<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Построить-модель-которая-будет-выбирать-топ-3-аэропорта-прилета-(вероятность-опоздания-минимальная-–-RMSE-метрика),-в-зависимости-от-аэропорта-вылета" data-toc-modified-id="Построить-модель-которая-будет-выбирать-топ-3-аэропорта-прилета-(вероятность-опоздания-минимальная-–-RMSE-метрика),-в-зависимости-от-аэропорта-вылета-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика), в зависимости от аэропорта вылета</a></span><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Выбор-параметров-и-получение-результата" data-toc-modified-id="Выбор-параметров-и-получение-результата-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Выбор параметров и получение результата</a></span></li></ul></li></ul></div>

[Данные - https://www.kaggle.com/usdot/flight-delays  ](https://www.kaggle.com/usdot/flight-delays) 

Описание - Сводная информация о количестве своевременных, задержанных, отмененных и измененных рейсов в США за 2015 год (5.8 млн записей)
Задачи:
1	Найти аэропорт с минимальной задержкой вылета 
2	Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport
3	Аэропорт с самой большой долей руления на 1 самолетовылет
4	Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика),  в зависимости от аэропорта вылета  

# Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика), в зависимости от аэропорта вылета

## Подготовка данных 

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [2]:
%%time
airlines= pd.read_csv('data/airlines.csv')
airports= pd.read_csv('data/airports.csv')
flights= pd.read_csv('data/flights.csv',dtype={'ORIGIN_AIRPORT':str,'DESTINATION_AIRPORT':str},nrows=2500000)

Wall time: 8.56 s


In [3]:
%%time
flights1=flights.merge(airports[['IATA_CODE','AIRPORT']], how='inner', left_on='ORIGIN_AIRPORT', right_on='IATA_CODE').drop (['IATA_CODE'], axis = 1)

Wall time: 16.9 s


In [4]:
%%time
flights2=flights1.merge(airports[['IATA_CODE','AIRPORT']], how='inner', left_on='DESTINATION_AIRPORT', right_on='IATA_CODE').drop (['IATA_CODE',], axis = 1)

Wall time: 20.2 s
Parser   : 331 ms


In [5]:
%%time
flights3=flights2.merge(airlines, how='inner', left_on='AIRLINE', right_on='IATA_CODE').drop (['IATA_CODE'], axis = 1)

Wall time: 5.8 s
Parser   : 118 ms


In [6]:
%%time
flights_real= flights3[~flights3.DIVERTED.isin([1])& ~flights3.CANCELLED.isin([1])].copy()

Wall time: 7.55 s
Parser   : 123 ms


In [7]:
def make_date(df):
 return dt.date(df.YEAR,df.MONTH,df.DAY)

In [8]:
%%time
flights_real['dDate']=flights_real.apply(make_date, axis=1)

Wall time: 5min 23s


In [9]:
%%time
flight_grouped = flights_real.groupby(['dDate','DESTINATION_AIRPORT'])['ARRIVAL_DELAY'].sum().reset_index()
flight_grouped['dDate'] = pd.to_datetime(flight_grouped['dDate'])

Wall time: 11.8 s
Compiler : 311 ms
Parser   : 541 ms


In [10]:
def make_features(data, max_lag, rolling_mean_size):
    data['YEAR'] =data.dDate.dt.year
    data['MONTH'] =data.dDate.dt.month
    data['DAY'] =data.dDate.dt.day
    data['DAYOFWEEK'] =data.dDate.dt.dayofweek
    data['is_weekend'] = data.DAYOFWEEK.isin([5,6])*1
    for lag in range (1, max_lag+1):
        data['ARRIVAL_DELAY_{}'.format(lag)] = data['ARRIVAL_DELAY'].shift(lag)     

In [11]:
%%time
DESTINATION = []
for dest in flight_grouped[flight_grouped.ARRIVAL_DELAY>=0].DESTINATION_AIRPORT.unique():
    tempo = flight_grouped[(flight_grouped.DESTINATION_AIRPORT==dest)&(flight_grouped.ARRIVAL_DELAY>=0)][['dDate','ARRIVAL_DELAY']]
    tempo.columns = ['dDate','ARRIVAL_DELAY']
    tempo = pd.DataFrame(tempo)
    
    try:
    
        make_features(tempo,5,2)
        tempo.dropna(inplace=True)
        tempo.set_index('dDate',inplace=True)
           
        X_train,X_test,y_train, y_test = train_test_split(tempo.drop('ARRIVAL_DELAY', axis=1),tempo.ARRIVAL_DELAY, shuffle=False, test_size=0.1)

        model_lr = LinearRegression()
        model_lr.fit(X_train,y_train)

        y_predicted_lr = model_lr.predict(X_test)

        DESTINATION.append([dest,y_test.mean(),np.sqrt(mean_squared_error(y_test, y_predicted_lr))])

    except Exception as e:
        print('Error', str(e))

Error With n_samples=0, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
Error With n_samples=1, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
Error With n_samples=0, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
Error With n_samples=0, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
Wall time: 28.8 s
Parser   : 420 ms


In [12]:
%%time
DESTINATION = pd.DataFrame(DESTINATION)
DESTINATION.columns = ['DESTINATION_AIRPORT','MEAN_ARRIVAL_DELAY_IN_PAST','RMSE']

Wall time: 132 ms


In [13]:
DESTINATION.head()

,DESTINATION_AIRPORT,MEAN_ARRIVAL_DELAY_IN_PAST,RMSE
0,ABQ,412.166667,273.292489
1,ACT,120.400000,113.634538
2,ACV,76.875000,61.212240
3,ACY,285.363636,248.995702
4,AEX,111.100000,86.982153


## Выбор параметров и получение результата

In [44]:
%%time
start_airport = np.random.choice(flights_real['ORIGIN_AIRPORT'].unique()) 
where_to_go_from_start_airport = flights_real[flights_real.ORIGIN_AIRPORT==start_airport]['DESTINATION_AIRPORT'].unique()

where_to_go_from_start_airport = pd.DataFrame(where_to_go_from_start_airport)
where_to_go_from_start_airport.columns = ['DESTINATION_AIRPORT']

top3 = where_to_go_from_start_airport\
    .merge(DESTINATION,on='DESTINATION_AIRPORT',how='inner')\
    .sort_values(by=['RMSE','MEAN_ARRIVAL_DELAY_IN_PAST'],ascending=[True, True])\
    .head(3)

print('3 лучших аэропорта прилета для аэропорта вылета',start_airport)
print('')
print(top3)

3 лучших аэропорта прилета для аэропорта вылета MSP

    DESTINATION_AIRPORT  MEAN_ARRIVAL_DELAY_IN_PAST       RMSE
103                 ABR                   17.666667   5.883368
100                 IMT                   30.666667  19.978529
70                  FCA                   42.666667  25.653165
Wall time: 490 ms
